In [1]:
import pickle
from typing import List

from nltk.tokenize import RegexpTokenizer
import numpy as np
from tqdm import tqdm

In [2]:
class BigramModel:
    def __init__(self):
        self.tokenizer = RegexpTokenizer(r'\w+')

        self.vocab = set()
        self.occur_dict = {}
        self.freq_dict = {}
        
    @property
    def vocab_size(self):
        return len(self.vocab)
        
    def _tokenize(self, texts: List[str]):
        result = []
        for text in texts:
            tokens = self.tokenizer.tokenize(text.lower())
            result.extend(tokens)
            
        return result
    
    def fit(self, data: List[str]):
        tokens = self._tokenize(data)
        self.vocab = set(tokens)

        vocab_dict = {v: 1 for v in self.vocab}
        occur_dict = {v: vocab_dict for v in self.vocab}
        freq_dict = {v: vocab_dict for v in self.vocab}
        
        for i in tqdm(range(0, len(tokens)-1)):
            w_1, w_2 = tokens[i], tokens[i+1]
            occur_dict[w_1][w_2] += 1
                
        c_w = {v: sum(occur_dict[v].values()) + self.vocab_size for v in tqdm(self.vocab)}
        for v in tqdm(self.vocab):
            v_c_w = c_w[v]
            for b in self.vocab:
                freq_dict[v][b] = occur_dict[v][b] / v_c_w
        
        self.occur_dict = occur_dict
        self.freq_dict = freq_dict
        
    def sentence_prob(self, sentence: str) -> float:
        tokens = self._tokenize([sentence])
        
        probs = []
        for i in range(0, len(tokens)-1, 2):
            bigram_prob = np.log(self.freq_dict[tokens[i][tokens[i+1]]])
            probs.append(bigram_prob)

        return sum(probs)
    
    def top_freq(self, word: str, count: int = 10):
        top = sorted(
            self.freq_dict[word].items(), key=lambda x: x[1], reverse=True
        )

        return top[:count]
    
    def predict(self, sentence: str, count: int):
        new_sentence = sentence
        for _ in range(count):
            last_word = new_sentence.split(' ')[-1]
            words, probs = self.freq_dict[last_word].items
            word = np.random.choice(words, p=probs)
            new_sentence += " " + word
            
        return new_sentence

In [3]:
with open('data/all_news.pkl', 'rb') as f:
    news = pickle.load(f)
data = [topic['body'] for topic in news]

In [4]:
model = BigramModel()

In [5]:
model.fit(data)

100%|████████████████████████████████| 190719/190719 [4:00:25<00:00, 13.22it/s]


In [9]:
vocab = model.vocab
occur_dict = model.occur_dict
freq_dict = model.freq_dict

In [14]:
occur_dict['я']

{'награжденныи': 0.0,
 'профсоюзных': 0.0,
 'гипергликемическии': 0.0,
 'канадский': 0.0,
 'обиталищ': 0.0,
 'афише': 0.0,
 'mdogg_dubcnn': 0.0,
 'военную': 0.0,
 'предлагающий': 0.0,
 'respekt': 0.0,
 'полоцке': 0.0,
 'микробов': 0.0,
 'коктейльный': 0.0,
 'сюрреализма': 0.0,
 'йик': 0.0,
 'купит': 0.0,
 'бескрайней': 0.0,
 'девятом': 0.0,
 'выходящие': 0.0,
 'фуна': 0.0,
 'распределила': 0.0,
 'погружения': 0.0,
 'чешскую': 0.0,
 'фиктивной': 0.0,
 'миротворцу': 0.0,
 'примененных': 0.0,
 '1930': 0.0,
 'yeah': 0.0,
 'капитулировали': 0.0,
 'монастыре': 0.0,
 'вштырило': 0.0,
 'предсмертной': 0.0,
 'реэкспорт': 0.0,
 'монастырских': 0.0,
 'проработанных': 0.0,
 'позорище': 0.0,
 'аранжировщиками': 0.0,
 'ошеломляет': 0.0,
 'использоваться': 0.0,
 'rivet': 0.0,
 'керамическая': 0.0,
 'обывателям': 0.0,
 'трудозатраты': 0.0,
 'самостоятельных': 0.0,
 'ввязываются': 0.0,
 'абсурдны': 0.0,
 'главнои': 0.0,
 'мостранспроект': 0.0,
 'соотечественников': 0.0,
 'фабиньо': 0.0,
 'позируют': 0.

In [6]:
model.sentence_prob("сегодня я увидел кактус")

TypeError: string indices must be integers

In [15]:
model.top_freq("я")

[('награжденныи', 0.0),
 ('профсоюзных', 0.0),
 ('гипергликемическии', 0.0),
 ('канадский', 0.0),
 ('обиталищ', 0.0),
 ('афише', 0.0),
 ('mdogg_dubcnn', 0.0),
 ('военную', 0.0),
 ('предлагающий', 0.0),
 ('respekt', 0.0)]

In [8]:
model.predict("сегодня я увидел кактус", 10)

AttributeError: 'str' object has no attribute 'copy'